### EXTRAÇÃO DOS DADOS 
Processo de conexão e requisição via <i>API</i>, a requisição foi dividia em dois blocos: <hr>

In [1]:
import pandas as pd
import requests

###### 1 - REQUISIÇÃO LISTA DE DEPUTADOS

In [2]:
url_requisicao_deputados = "https://dadosabertos.camara.leg.br/api/v2/deputados"
resultado = requests.get(url_requisicao_deputados)
deputados = pd.DataFrame(resultado.json()['dados'])

lista_exclusao_deputados = ['uri', 'uriPartido', 'idLegislatura', 'urlFoto']
i = 0

while i < len(lista_exclusao_deputados):
    del deputados[lista_exclusao_deputados[i]] 
    i += 1


deputados.nome = [str(nome).upper() for nome in deputados.nome]
deputados.email = [str(email).upper() for email in deputados.email]

deputados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            513 non-null    int64 
 1   nome          513 non-null    object
 2   siglaPartido  513 non-null    object
 3   siglaUf       513 non-null    object
 4   email         513 non-null    object
dtypes: int64(1), object(4)
memory usage: 12.1+ KB


###### 2 - REQUISIÇÃO LISTA DE GASTOS

In [3]:
gastos = []
for id in deputados.id:
    url_despesa = "https://dadosabertos.camara.leg.br/api/v2/deputados/"
    url_despesa = url_despesa +str(id)+"/despesas?ordem=ASC&ano=2020&itens=999999"
    resposta = requests.get(url_despesa)
    gasto = pd.DataFrame(resposta.json()['dados'])
    gasto['id'] = id
    gastos.append(gasto)
    
total_gastos = pd.concat(gastos)
lista_exclusao_gastos = ['numRessarcimento','dataDocumento','codDocumento', 'codTipoDocumento','numDocumento', 'urlDocumento','codLote', 'parcela']
i = 0

while i < len(lista_exclusao_gastos):
    del total_gastos[lista_exclusao_gastos[i]] 
    i += 1
    
total_gastos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35739 entries, 0 to 99
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ano                35739 non-null  float64
 1   mes                35739 non-null  float64
 2   tipoDespesa        35739 non-null  object 
 3   tipoDocumento      35739 non-null  object 
 4   valorDocumento     35739 non-null  float64
 5   nomeFornecedor     35739 non-null  object 
 6   cnpjCpfFornecedor  35739 non-null  object 
 7   valorLiquido       35739 non-null  float64
 8   valorGlosa         35739 non-null  float64
 9   id                 35739 non-null  int64  
dtypes: float64(5), int64(1), object(4)
memory usage: 2.5+ MB


### TRATAMENTO DOS DADOS 
Modelagem dos dados da requisição 'Lista de Gastos'. <hr>

<strong>Informações detalhadas sobre a lista de gastos:<strong> <br> 

In [4]:
total_gastos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35739 entries, 0 to 99
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ano                35739 non-null  float64
 1   mes                35739 non-null  float64
 2   tipoDespesa        35739 non-null  object 
 3   tipoDocumento      35739 non-null  object 
 4   valorDocumento     35739 non-null  float64
 5   nomeFornecedor     35739 non-null  object 
 6   cnpjCpfFornecedor  35739 non-null  object 
 7   valorLiquido       35739 non-null  float64
 8   valorGlosa         35739 non-null  float64
 9   id                 35739 non-null  int64  
dtypes: float64(5), int64(1), object(4)
memory usage: 2.5+ MB


Observações:
- Aqui já conseguimos ver se existe diferença entre o count das colunas, sendo possível verificar em quais existem valores nulos. 
- Neste caso não houve a necessidade de tratamento para o tipo dos dados, pois, todos estão retornando de forma correta: <br> Textos em Object/String e Valores calculáveis em Float.

<strong>Verificação e correção dos dados nulos ou duplicados: <strong> <br>

In [5]:
print('LISTAGEM DE COLUNAS COM VALORES NULOS:')
print(total_gastos.isnull().sum())
print('TOTAL DE DADOS VS COLUNAS:')
print(total_gastos.shape)
print('TOTAL DE DADOS VS COLUNAS SEM DUPLICAÇÕES:')
total_gastos.drop_duplicates()
print(total_gastos.shape)

LISTAGEM DE COLUNAS COM VALORES NULOS:
ano                  0
mes                  0
tipoDespesa          0
tipoDocumento        0
valorDocumento       0
nomeFornecedor       0
cnpjCpfFornecedor    0
valorLiquido         0
valorGlosa           0
id                   0
dtype: int64
TOTAL DE DADOS VS COLUNAS:
(35739, 10)
TOTAL DE DADOS VS COLUNAS SEM DUPLICAÇÕES:
(35739, 10)


Observações:
- Neste caso não houve necessidade de ajustes pois não há nenhum dado nulo ou duplicado. <br> 
É possível identificar através da contagem de valores nulos = 0 e também na diferença na contagem dos dados vs colunas com ou sem duplicações.

<strong>Analisando a coluna Fornecedores:<strong> <br> 

In [6]:
print('DADOS FORNECEDORES SEM ALTERAÇÃO:')
print(total_gastos.nomeFornecedor.value_counts().head(15))
print('----------------------------------------------')
print('DADOS FORNECEDORES AJUSTADOS:')
total_gastos.nomeFornecedor = [str(fornecedor).upper().replace("S.A.", "S.A").replace("S/A","S.A").replace("LTDA-ME","LTDA") for fornecedor in total_gastos.nomeFornecedor]
print(total_gastos.nomeFornecedor.value_counts().head(15))

DADOS FORNECEDORES SEM ALTERAÇÃO:
RAMAL                                               1394
CASCOL COMBUSTIVEIS PARA VEICULOS LTDA               941
CORREIOS - SEDEX CONVENCIONAL                        873
Telefônica Brasil S.A. VIVO                          723
Cia Aérea - TAM                                      709
CELULAR FUNCIONAL                                    579
UBER DO BRASIL TECNOLOGIA LTDA.                      459
Claro S.A                                            377
CORREIOS - CARTA COMERCIAL                           332
CORREIOS - ENCOMENDA PAC                             292
FRATELLI POSTO DE COMBUSTIVEIS LTDA                  248
SERVICO NACIONAL DE APRENDIZAGEM COMERCIAL SENAC     243
WMS COMERCIO DE ARTIGOS DE PAPELARIA LTDA-ME         187
PANTANAL VEÍCULOS LTDA                               170
LATAM AIRLINES                                       150
Name: nomeFornecedor, dtype: int64
----------------------------------------------
DADOS FORNECEDORES AJUSTADOS:

Observações:
- Identifiquei que o dado não está padronizado, há alguns com escrita em caixa alta e outros não. Também contém variações de escrita para o mesmo fornecedor.

<strong>Analisando a coluna Tipos de Despesas:<strong> <br> 

In [8]:
print('DADOS FORNECEDORES SEM ALTERAÇÃO:')
print(total_gastos.tipoDespesa.value_counts().head(20))
print('----------------------------------------------')
print('DADOS FORNECEDORES AJUSTADOS:')
total_gastos.tipoDespesa = [str(despesas).upper() for despesas in total_gastos.tipoDespesa]
print(total_gastos.tipoDespesa.value_counts().head(20))

DADOS FORNECEDORES SEM ALTERAÇÃO:
COMBUSTÍVEIS E LUBRIFICANTES.                                12783
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR     6555
TELEFONIA                                                     4127
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                          3187
SERVIÇOS POSTAIS                                              2504
SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO                     1411
LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES                 1238
FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                    1101
EMISSÃO BILHETE AÉREO                                          955
CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.                  622
HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.         518
PASSAGENS AÉREAS                                               342
SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.       168
ASSINATURA DE PUBLICAÇÕES                                      134
PASSAGENS TERRESTRES, MARÍTI

Observações:
- Identifiquei que nessa coluna o dado também não está padronizado contém alguns com escrita em caixa alta e outros não. 

<strong>Analisando a colunas ano e mês:<strong> <br> 

In [10]:
total_gastos.ano = [str(ano_list).replace("2020.0", "2020") for ano_list in total_gastos.ano]
total_gastos.mes = [str(mes_list).replace("1.0", "1").replace("2.0", "2").replace("3.0", "3").replace("4.0", "4") for mes_list in total_gastos.mes]

Observações:
- Nessa variável fiz um ajuste para 'string' e retirei as casas decimais. 

Para os demais dados da requisição despesas não houve necessidade de ajustes:
- valorDocumento
- cnpjCpfFornecedor
- valorLiquido
- valorGlosa
- id

<strong>Renomeando o nome das colunas:<strong> <br> 

In [11]:
deputados.columns = ['ID', 'NOME', 'PARTIDO','UF','E-MAIL']
deputados.head()

,ID,NOME,PARTIDO,UF,E-MAIL
0,204554,ABÍLIO SANTANA,PL,BA,DEP.ABILIOSANTANA@CAMARA.LEG.BR
1,204521,ABOU ANNI,PSL,SP,DEP.ABOUANNI@CAMARA.LEG.BR
2,204379,ACÁCIO FAVACHO,PROS,AP,DEP.ACACIOFAVACHO@CAMARA.LEG.BR
3,204560,ADOLFO VIANA,PSDB,BA,DEP.ADOLFOVIANA@CAMARA.LEG.BR
4,204528,ADRIANA VENTURA,NOVO,SP,DEP.ADRIANAVENTURA@CAMARA.LEG.BR


In [12]:
total_gastos.columns = ['ANO', 'MÊS', 'TIPO-DESPESA','TIPO-DOCUMENTO','VALOR-DOCUMENTO','NOME-FORNECEDOR','CNPJ-FORNECEDOR','VALOR-LÍQUIDO','VALOR-GLOSA','ID']
total_gastos.head()

,ANO,MÊS,TIPO-DESPESA,TIPO-DOCUMENTO,VALOR-DOCUMENTO,NOME-FORNECEDOR,CNPJ-FORNECEDOR,VALOR-LÍQUIDO,VALOR-GLOSA,ID
0,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,132.76,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,118.35,14.41,204554
1,2020,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,162.27,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,159.97,2.30,204554
2,2020,3,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,284.36,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,281.03,3.33,204554
3,2020,4,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,80.96,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,80.96,0.00,204554
4,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Recibos/Outros,1500.00,MATHEUS QUADROS LOIOLA MARTINS,06540941528,1500.00,0.00,204554


### CRIANDO UMA BASE COM OS DADOS AJUSTADOS
Criação de um banco de dados SQLite que salva todos os dados já tratados para análises sem a necessidade de requisições na API.
<hr>

In [15]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///dados_abertos.db')
deputados.to_sql('deputados', engine)
total_gastos.to_sql('gastos', engine)

In [16]:
deputados = pd.read_sql("select * from deputados;", engine)
deputados.head()

,index,ID,NOME,PARTIDO,UF,E-MAIL
0,0,204554,ABÍLIO SANTANA,PL,BA,DEP.ABILIOSANTANA@CAMARA.LEG.BR
1,1,204521,ABOU ANNI,PSL,SP,DEP.ABOUANNI@CAMARA.LEG.BR
2,2,204379,ACÁCIO FAVACHO,PROS,AP,DEP.ACACIOFAVACHO@CAMARA.LEG.BR
3,3,204560,ADOLFO VIANA,PSDB,BA,DEP.ADOLFOVIANA@CAMARA.LEG.BR
4,4,204528,ADRIANA VENTURA,NOVO,SP,DEP.ADRIANAVENTURA@CAMARA.LEG.BR


In [17]:
total_gastos = pd.read_sql("select * from gastos;", engine)
total_gastos.head()

,index,ANO,MÊS,TIPO-DESPESA,TIPO-DOCUMENTO,VALOR-DOCUMENTO,NOME-FORNECEDOR,CNPJ-FORNECEDOR,VALOR-LÍQUIDO,VALOR-GLOSA,ID
0,0,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,132.76,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,118.35,14.41,204554
1,1,2020,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,162.27,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,159.97,2.30,204554
2,2,2020,3,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,284.36,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,281.03,3.33,204554
3,3,2020,4,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,80.96,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,80.96,0.00,204554
4,4,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Recibos/Outros,1500.00,MATHEUS QUADROS LOIOLA MARTINS,06540941528,1500.00,0.00,204554
